<a href="https://colab.research.google.com/github/deductiveclouds/Tensorflow/blob/main/%5BGK%5D_05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!black [GK]_05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb

UsageError: Line magic function `%nb_black` not found.


<IPython.core.display.Javascript object>

In [ ]:
# Preliminary setup for new runtime
# !pip uninstall -y tensorflow
# !pip install tensorflow=='2.15.0'
# !pip freeze | grep tensorflow
# !sudo apt install nvidia-utils-560

In [ ]:
!nvidia-smi

In [25]:
# Get helper_functions.py from github
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import functions to use from above file
from helper_functions import (create_tensorboard_callback,plot_loss_curves, unzip_data,walk_through_dir,)

<IPython.core.display.Javascript object>

In [ ]:
# # Get 10% of the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# # Unzip the data
unzip_data("10_food_classes_10_percent.zip")

In [ ]:
# Walk through the directory for 10 percent
walk_through_dir("10_food_classes_10_percent")

In [ ]:
# Create path names to directories for 10 percent
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test"

In [ ]:
# Create input data for 10 percent
import tensorflow as tf

IMAGE_SIZE = (224, 224, 3)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMAGE_SIZE[:2],
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir, image_size=IMAGE_SIZE[:2], label_mode="categorical"
)

In [ ]:
# Check out the data
train_data_10_percent

In [ ]:
# Check out the labels
test_data_10_percent.class_names

In [ ]:
# See contents of a batch of data
import random

batch_num = random.randint(0, len(train_data_10_percent))
for index, items in enumerate(train_data_10_percent.take(batch_num)):
    print(index, items[0].shape, items[1].shape)

In [ ]:
# Create and fit the baseline model_0
from os import name

base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=IMAGE_SIZE, name="input_layer")
x = base_model(inputs)

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer_2d")(x)

outputs = tf.keras.layers.Dense(units=10, activation="softmax", name="output_layer")(x)

model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

history_model_0 = model_0.fit(
    x=train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data_10_percent,
    validation_steps=len(test_data_10_percent),
    callbacks=[
        create_tensorboard_callback(
            dir_name="transfer_learning", experiment_name="model_0"
        )
    ],
)

In [ ]:
# Check based model layers
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)

In [ ]:
base_model.summary()

In [ ]:
model_0.summary()

In [ ]:
plot_loss_curves(history_model_0)

In [ ]:
# Create 1 percent of the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

unzip_data("10_food_classes_1_percent.zip")

In [ ]:
# Create path names to directories for 1 percent
train_dir = "10_food_classes_1_percent/train/"
test_dir = "10_food_classes_1_percent/test/"

In [ ]:
# Walk through the directory for 10 percent
walk_through_dir("10_food_classes_1_percent")

In [ ]:
# Create input data for 10 percent
train_data_1_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMAGE_SIZE[:2],
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)
test_data_1_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir, image_size=IMAGE_SIZE[:2], label_mode="categorical"
)

In [ ]:
# Create data augmentation layer
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip(mode="horizontal"),
        tf.keras.layers.RandomRotation(factor=0.2),
        tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2),
        tf.keras.layers.RandomHeight(factor=0.2),
        tf.keras.layers.RandomWidth(factor=0.2),
    ],
    name="data_augmentation",
)

In [ ]:
# Create and fit the 1 percent with data augmentation model_1
inputs = tf.keras.layers.Input(shape=IMAGE_SIZE, name="input_layer")

x = data_augmentation(inputs)

base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False

x = base_model(x, training=False)

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

outputs = tf.keras.layers.Dense(units=10, activation="softmax", name="output_layer")(x)

model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

history_model_1 = model_1.fit(
    x=train_data_1_percent,
    epochs=5,
    steps_per_epoch=len(train_data_1_percent),
    validation_data=test_data_1_percent,
    validation_steps=(0.25 * len(test_data_1_percent)),
    callbacks=[
        create_tensorboard_callback(
            dir_name="transfer_learning", experiment_name="model_1"
        )
    ],
)

In [ ]:
# Evaluate on the test data
results_1_percent_data_aug = model_1.evaluate(test_data_1_percent)
results_1_percent_data_aug

In [ ]:
plot_loss_curves(history_model_1)

In [ ]:
# Remove comment when starting a new runtime
import tensorflow as tf


def create_base_model(
    input_shape: tuple[int, int, int] = (224, 224, 3),
    output_shape: int = 10,
    learning_rate: float = 0.001,
    training: bool = False,
) -> tf.keras.Model:
    base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
        include_top=False
    )
    base_model.trainable = training
    inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer")
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
    outputs = tf.keras.layers.Dense(
        units=output_shape, activation="softmax", name="output_layer"
    )(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["accuracy"],
    )
    return model

In [ ]:
model_2 = create_base_model()
checkpoint_path = "ten_percent_model_checkpoint_weights/checkpoint.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
)
initial_epochs = 5
history_model_2 = model_2.fit(
    x=train_data_10_percent,
    epochs=initial_epochs,
    validation_data=test_data_10_percent,
    validation_steps=int(0.25 * len(test_data_10_percent)),
    callbacks=[
        create_tensorboard_callback("transfer_learning", "10_percent_data_aug"),
        checkpoint_callback,
    ],
)

In [ ]:
results_10_percent_data_aug = model_2.evaluate(test_data_10_percent)
results_10_percent_data_aug

In [ ]:
plot_loss_curves(history_model_2)

In [ ]:
model_2.load_weights(checkpoint_path)
loaded_weights_model_results = model_2.evaluate(test_data_10_percent)
results_10_percent_data_aug == loaded_weights_model_results

In [ ]:
# Remove comment when starting a new runtime
import numpy as np

np.isclose(results_10_percent_data_aug, loaded_weights_model_results)

In [ ]:
for layer_number, layer in enumerate(model_2.layers):
    print(
        f"Layer: {layer_number}, Layer name: {layer.name}, Trainable: {layer.trainable}"
    )

In [ ]:
model_3 = model_2
model_3_base_model = model_3.layers[2]
model_3_base_model.name
model_3_base_model.trainable = True
for layer in model_3_base_model.layers[:-10]:
    layer.trainable = False
model_3.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)
fine_tune_epochs = initial_epochs + 5
history_model_3 = model_3.fit(
    x=train_data_10_percent,
    epochs=fine_tune_epochs,
    steps_per_epoch=len(train_data_10_percent),
    initial_epoch=history_model_2.epoch[-1],
    validation_data=test_data_10_percent,
    validation_steps=int(0.25 * len(test_data_10_percent)),
    callbacks=[
        create_tensorboard_callback("transfer_learning", "10_percent_fine_tune_last_10")
    ],
)

In [ ]:
# Remove comment when starting a new runtime
import matplotlib.pyplot as plt


def compare_histories(original_history, new_history, initial_epochs=5):
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]
    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]
    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label="Training Accuracy")
    plt.plot(total_val_acc, label="Validation Accuracy")
    plt.plot(
        [initial_epochs - 1, initial_epochs - 1], plt.ylim(), label="Start Fine Tuning"
    )
    plt.legend(loc="lower right")
    plt.title("Training and Validation Accuracy")
    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label="Training Loss")
    plt.plot(total_val_loss, label="Validation Loss")
    plt.plot(
        [initial_epochs - 1, initial_epochs - 1], plt.ylim(), label="Start Fine Tuning"
    )
    plt.legend(loc="upper right")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epoch")
    plt.show()

In [ ]:
compare_histories(original_history=history_model_2, new_history=history_model_3)

In [ ]:
# Remove comment when starting a new runtime
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

train_dir_full = "10_food_classes_all_data/train/"
test_dir_full = "10_food_classes_all_data/test/"

In [ ]:
walk_through_dir("10_food_classes_all_data")

In [ ]:
train_data_full = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir_full,
    image_size=IMAGE_SIZE[:2],
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)
test_data_full = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir_full,
    image_size=IMAGE_SIZE[:2],
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)

In [ ]:
model_4 = create_base_model(learning_rate=0.0001)
model_4.load_weights(checkpoint_path)
model_4.evaluate(test_data_full)

In [ ]:
model_4_base_model = model_4.layers[2]
model_4_base_model.trainable = True
for layer in model_4_base_model.layers[:-10]:
    layer.trainable = False
model_4.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)
fine_tune_epochs = initial_epochs + 5
history_model_4 = model_4.fit(
    x=train_data_full,
    epochs=fine_tune_epochs,
    steps_per_epoch=len(train_data_full),
    initial_epoch=history_model_2.epoch[-1],
    validation_data=test_data_full,
    validation_steps=int(0.25 * len(test_data_full)),
    callbacks=[
        create_tensorboard_callback(
            "transfer_learning", "full_10_classes_fine_tune_last_10"
        )
    ],
)

In [ ]:
results_fine_tune_full_data = model_4.evaluate(test_data_full)
results_fine_tune_full_data

In [ ]:
compare_histories(history_model_2, history_model_4)